In [6]:
!git add portfolio.ipynb

In [8]:
!git commit -m "Add portfolio notebook"


[master (root-commit) 6181d53] Add portfolio notebook
 1 file changed, 76 insertions(+)
 create mode 100644 portfolio.ipynb


In [10]:
!git push origin master


fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
